In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

Lectura de .csv limpios

In [2]:
data = pd.read_csv("../../files/HR_RAW_DATA.csv", index_col = 0)

employees = pd.read_csv("../../files/df_exploracion_employees.csv" , index_col = 0)

experience = pd.read_csv("../../files/df_employee_exp.csv")

surveys = pd.read_csv("../../files/surveys.csv" , index_col = 0)

employment = pd.read_csv("transofrmacion_patri" )

salary = pd.read_csv("../../files/salarydetails_w_floats.csv" , index_col = 0)

hours = pd.read_csv("../../files/df_Salary_SH.csv")

Concatenación de archivos

In [3]:
df_limpio = pd.concat([data['employeenumber'], employees, experience, surveys, employment, salary, hours], axis = 1, ignore_index = False)

Homogeneización de columnas

In [4]:
nuevas_columnas = {'employeenumber' : 'EmployeeID' , 'NUMBERCHILDREN' : 'NumberChildren' , 'NUMCOMPANIESWORKED' : 'NumCompaniesWorked', 'TOTALWORKINGYEARS' : 'TotalWorkingYears' , 'YEARSWITHCURRMANAGER' : 'YearsWithCurrManager' , 'WORKLIFEBALANCE' : 'WorkLifeBalance'}

In [5]:
df_limpio.rename(columns = nuevas_columnas, inplace = True)

Limpieza de duplicados

In [6]:
df_limpio = df_limpio.drop_duplicates(keep="first")

In [7]:
df_limpio = df_limpio.reset_index()
df_limpio.drop("index", axis=1, inplace=True)

Revisión de aquellos empleados sin EmployeeID que puedan ser el mismo (coinciden en todas las columnas salvo RemoteWork)

In [8]:
# Saca un DataFrame de los empleados con EmployeeID nulo y que coinciden en todo salvo en RemoteWork

# Filtra las filas con valores nulos en 'EmployeeID'
filas_nulas = df_limpio['EmployeeID'].isnull()
df_filas_nulas = df_limpio[filas_nulas]

# Excluye la columna 'RemoteWork' para buscar duplicados
df_sin_remote = df_filas_nulas.drop(columns=['RemoteWork'])

# Busca duplicados en las filas filtradas
duplicados_sin_RW = df_sin_remote[df_sin_remote.duplicated(keep=False)]

# Extrae los indices
indices_duplicados = duplicados_sin_RW.index

# Crea una lista de los indices con las coincidencias
lista_indices_mismo_empleado = df_limpio.loc[indices_duplicados , :].sort_values(['Age' , 'Gender']).index

Imputación general de nulos para la columna EmployeeID

In [9]:
def asignar_id(valor, contador):
    
    if pd.isnull(valor):
        contador[0] += 1  # Incrementar el contador si el valor es NaN
        id_nulo = f'nan{contador[0]}'  # Devolver el ID con el contador actualizado
        return id_nulo
    
    else:
        return valor.replace(',0' , '')  # Devolver el valor quitándole los decimales

# Reiniciar el contador cada vez que se aplique la función
def aplicar_asignar_id(columna):
    contador = [0]
    return columna.apply(lambda x: asignar_id(x, contador))

# Aplicar la función a la columna 'EmployeeID' y reasignar los resultados
df_limpio['EmployeeID'] = aplicar_asignar_id(df_limpio['EmployeeID'])


Asignación de la misma EmployeeID a los empleados que aparecen en `lista_indices_mismo_empleado`

In [10]:
indice_sublista = 0

for indice in lista_indices_mismo_empleado[1::2]:
    
    dato = lista_indices_mismo_empleado[indice_sublista]
    
    df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]
    
    indice_sublista += 2

/tmp/ipykernel_6872/1808907360.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['EmployeeID'][indice] = df_limpio['EmployeeID'][dato]


In [15]:
df_limpio.to_csv("../../files/df_to_SQL.csv")